In [ ]:
import numpy as np 
import pandas as pd 
from transformers import default_data_collator, Trainer
from transformers import AutoTokenizer, TrainingArguments,AutoModelForQuestionAnswering
import tensorflow as tf
from datasets import Dataset

In [ ]:
df = pd.read_csv('CleanedDataset.csv')
id=[]
for i in range(1,354):
    id.append(i)
df["id"]=id

In [ ]:
train = df
train.head()

,Unnamed: 0,context,question,answer_text,answer_start,id
0,0.0,సాధారణ వయోజన మానవుని అస్థిపంజరం కింది 206 (స్ట...,మానవ శరీరంలో ఎన్ని ఎముకలు ఉంటాయి?,206,38.0,1
1,2.0,"సర్ అలెగ్జాండర్ ఫ్లెమింగ్ (ఆగస్టు 6, 1881 - మా...",పెన్సిలిన్‌ను ఎవరు కనుగొన్నారు?,సర్ అలెగ్జాండర్ ఫ్లెమింగ్,0.0,2
2,3.0,లాలిపాట అనేది పసిపాప ఏడుపు ఆపడానికి మరియు నిద్...,తమిళనాడులో పిల్లలను నిద్రపుచ్చడానికి పాడే పాటన...,లాలిపాట,0.0,3
3,5.0,ఎలక్ట్రానిక్స్ అనేది ఎలక్ట్రాన్లు లేదా ఎలక్ట్ర...,త్రిశూలాలను ఉపయోగించిన మొదటి కంపెనీ ఏది?,IBM,3819.0,4
4,6.0,"అలెగ్జాండర్ గ్రాహం బెల్ (మార్చి 3, 1847 - ఆగష్...",ఫోన్‌ని ఎవరు కనుగొన్నారు?,అలెగ్జాండర్ గ్రాహం బెల్,0.0,5


In [ ]:
test = pd.read_csv('newtest1.csv')
test.tail()

,id,context,question
2,d60987e0e,గుస్తావ్ రాబర్ట్ కిర్చోఫ్ (12 మార్చి 1824 - 17...,గుస్తావ్ కిర్చోఫ్ ఎప్పుడు జన్మించాడు?
3,f99c770dc,అల్యూమినియం (ఇంగ్లీష్: అల్యూమినియం; ఉత్తర అమెర...,అల్యూమినియం పరమాణు సంఖ్య ఎంత?
4,40dec1964,సహకార ఉద్యమ చరిత్ర ఇంగ్లాండ్‌లో మొట్టమొదటి సహక...,భారతదేశంలో హరిత విప్లవ పితామహుడిగా ఎవరు పరిగణి...
5,307200156,ఆవుకు 4 కాళ్లు ఉంటాయి,ఆవుకు ఎన్ని కాళ్లు ఉన్నాయి?
6,39dec1965,పవన్ కళ్యాణ్ (జననం కొణిదెల కళ్యాణ్ బాబు;[1] 2 ...,పవన్ కళ్యాణ్ ఎవరు?


In [ ]:
train.drop(train.columns[0],axis=1,inplace=True)
train.dropna(inplace=True)

In [ ]:
train.shape

(337, 5)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("deepset/xlm-roberta-large-squad2")

In [ ]:
batch_size = 4
max_length = 384 
doc_stride = 128
pad_on_right = tokenizer.padding_side == "right"

In [ ]:
def prepare_train_features(examples):
    
    examples["question"] = [q.lstrip() for q in examples["question"]]

   
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    # The offset mappings will give us a map from token to character position in the original context. This will
    # help us compute the start_positions and end_positions.
    offset_mapping = tokenized_examples.pop("offset_mapping")

    # Let's label those examples!
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        # We will label impossible answers with the index of the CLS token.
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        answers = examples["answers"][sample_index]
        # If no answers are given, set the cls_index as answer.
        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            # Start/end character index of the answer in the text.
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])

            # Start token index of the current span in the text.
            token_start_index = 0
            while sequence_ids[token_start_index] != (1 if pad_on_right else 0):
                token_start_index += 1

            # End token index of the current span in the text.
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != (1 if pad_on_right else 0):
                token_end_index -= 1

            # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # Otherwise move the token_start_index and token_end_index to the two ends of the answer.
                # Note: we could go after the last offset if the answer is the last word (edge case).
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples

In [ ]:
def convert_answers(r):
    start = r[0]
    text = r[1]
    return {
        'answer_start': [start],
        'text': [text]
    }

train = train.sample(frac=1, random_state=42)
train['answers'] = train[['answer_start', 'answer_text']].apply(convert_answers, axis=1)

df_train = train[:-64].reset_index(drop=True)
df_valid = train[-64:].reset_index(drop=True)

train_dataset = Dataset.from_pandas(df_train)
valid_dataset = Dataset.from_pandas(df_valid)

In [ ]:
train_dataset[0]

{'context': 'ఏనుగు మొక్కలను తినే (ఆకులను తినే) క్షీరదం. ఇది అన్ని భూమి జంతువులలో అతిపెద్దది. చాలా రోజులు (~ 70 సంవత్సరాలు) జీవించడం కూడా సాధ్యమే. ఇది మానవుల కంటే ఎక్కువ కాలం జీవించే భూగోళ జంతువు. ఏనుగులు చాలా బలమైనవి. మాంసాహారులు మరియు వన్యప్రాణుల ఆహార శ్రేణిలో అధికంగా ఉండే సింహాలు (అరిమా), పులులు మొదలైనవి అపారమైన బలాన్ని కలిగి ఉంటాయి. సింహాలు కలిసి అలసిపోయిన లేదా ఒంటరిగా వచ్చే ఏనుగును చంపేస్తాయి. అయితే, ఇటువంటి కేసులు చాలా అరుదు. ఏనుగులు కుటుంబ సమేతంగా జీవిస్తాయి. అలాగే వీటికి జ్ఞాపక శక్తి ఎక్కువ. ప్రస్తుతం ప్రపంచంలో మూడు రకాల ఏనుగులు మిగిలి ఉన్నాయి. అవి ఆఫ్రికన్ బుష్ ఏనుగులు, ఆఫ్రికన్ అడవి ఏనుగులు మరియు ఆసియా ఏనుగులు. వీటి మధ్య ప్రత్యేక తేడాలున్నాయి. సాధారణంగా, ఏనుగులన్నీ దాదాపు 70 ఏళ్లు జీవిస్తాయి. మగ ఏనుగును కలిరు అంటారు. ఆడ ఏనుగు పేరు పెట్టండి. ఏనుగు పిల్లను దూడ లేదా దూడ అంటారు. ఏనుగు బిగ్గరగా లేచిన శబ్దం యొక్క విభజన. ఏనుగులు ఆఫ్రికన్ ఏనుగులు ఆసియా ఏనుగుల కంటే పెద్దవి. పెద్ద చెవి లోబ్స్ కలిగి ఉండండి. ఆఫ్రికన్ ఏనుగులకు మగ మరియు ఆడ దంతాలు ఉంటాయి. కానీ ఆసియా ఆడ ఏనుగులలో దంతాలు చాలా అ

In [ ]:
tokenized_train_ds = train_dataset.map(prepare_train_features, batched=True, remove_columns=train_dataset.column_names)
tokenized_valid_ds = valid_dataset.map(prepare_train_features, batched=True, remove_columns=train_dataset.column_names)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
%env WANDB_DISABLED=True
args = TrainingArguments(
    f"chaii-qa",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=3e-5,
    warmup_ratio=0.1,
    gradient_accumulation_steps=8,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=1,
    weight_decay=0.01,
)

Using the `WAND_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


env: WANDB_DISABLED=True


In [ ]:
data_collator = default_data_collator

model = AutoModelForQuestionAnswering.from_pretrained('deepset/xlm-roberta-large-squad2')

trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_train_ds,
    eval_dataset=tokenized_valid_ds,
    data_collator=data_collator,
    tokenizer=tokenizer,
)

In [ ]:
#trainer.train()
#trainer.save_model("teluguQAtrained")

In [ ]:
def prepare_validation_features(examples):
    # Some of the questions have lots of whitespace on the left, which is not useful and will make the
    # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
    # left whitespace
    examples["question"] = [q.lstrip() for q in examples["question"]]

    # Tokenize our examples with truncation and maybe padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")

    # We keep the example_id that gave us this feature and we will store the offset mappings.
    tokenized_examples["example_id"] = []

    for i in range(len(tokenized_examples["input_ids"])):
        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)
        context_index = 1 if pad_on_right else 0

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        tokenized_examples["example_id"].append(examples["id"][sample_index])

        # Set to None the offset_mapping that are not part of the context so it's easy to determine if a token
        # position is part of the context or not.
        tokenized_examples["offset_mapping"][i] = [
            (o if sequence_ids[k] == context_index else None)
            for k, o in enumerate(tokenized_examples["offset_mapping"][i])
        ]

    return tokenized_examples

In [ ]:
validation_features = valid_dataset.map(prepare_validation_features, batched=True, remove_columns=valid_dataset.column_names)
validation_features


  0%|          | 0/1 [00:00<?, ?ba/s]

Dataset({
    features: ['attention_mask', 'example_id', 'input_ids', 'offset_mapping'],
    num_rows: 744
})

In [ ]:
valid_feats_small = validation_features.map(lambda example: example, remove_columns=['example_id', 'offset_mapping'])
valid_feats_small

  0%|          | 0/744 [00:00<?, ?ex/s]

Dataset({
    features: ['attention_mask', 'input_ids'],
    num_rows: 744
})

In [ ]:
%%time
raw_predictions = trainer.predict(valid_feats_small)

***** Running Prediction *****
  Num examples = 744
  Batch size = 4


Wall time: 40min 28s


In [ ]:
raw_predictions[0]

(array([[-1.7307278, -4.905709 , -8.091514 , ..., -5.151561 , -5.151561 ,
         -5.151561 ],
        [ 2.3581321, -6.60115  , -6.6683226, ..., -8.998268 , -7.078917 ,
         -5.0758367],
        [ 3.0882454, -4.4863253, -4.607616 , ..., -5.03899  , -5.4489303,
         -4.67996  ],
        ...,
        [ 1.9099542, -7.1700754, -6.898814 , ..., -6.091493 , -8.396437 ,
         -5.293714 ],
        [ 2.428232 , -6.615751 , -6.454987 , ..., -7.150067 , -7.3443346,
         -6.1944537],
        [ 2.9652946, -7.6203847, -6.9621773, ..., -6.6966796, -6.6966796,
         -6.6966796]], dtype=float32),
 array([[-3.5258548e+00, -8.5329723e+00, -9.3933372e+00, ...,
         -7.3164144e+00, -7.3164144e+00, -7.3164144e+00],
        [-7.8735780e-03, -1.0852720e+01, -1.0175138e+01, ...,
         -7.9691792e+00, -7.2174368e+00, -7.2697997e+00],
        [ 6.9269156e-01, -8.7058067e+00, -9.0190191e+00, ...,
         -7.4994249e+00, -5.9887919e+00, -6.9253769e+00],
        ...,
        [ 7.0243418e-

In [ ]:
max_answer_length = 30

In [ ]:
import collections

examples = valid_dataset
features = validation_features

example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
features_per_example = collections.defaultdict(list)
for i, feature in enumerate(features):
    features_per_example[example_id_to_index[feature["example_id"]]].append(i)

In [ ]:
from tqdm.auto import tqdm

def postprocess_qa_predictions(examples, features, raw_predictions, n_best_size = 20, max_answer_length = 30):
    all_start_logits, all_end_logits = raw_predictions
    # Build a map example to its corresponding features.
    example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
    features_per_example = collections.defaultdict(list)
    for i, feature in enumerate(features):
        features_per_example[example_id_to_index[feature["example_id"]]].append(i)

    # The dictionaries we have to fill.
    predictions = collections.OrderedDict()

    # Logging.
    print(f"Post-processing {len(examples)} example predictions split into {len(features)} features.")

    # Let's loop over all the examples!
    for example_index, example in enumerate(tqdm(examples)):
        # Those are the indices of the features associated to the current example.
        feature_indices = features_per_example[example_index]

        min_null_score = None # Only used if squad_v2 is True.
        valid_answers = []
        
        context = example["context"]
        # Looping through all the features associated to the current example.
        for feature_index in feature_indices:
            # We grab the predictions of the model for this feature.
            start_logits = all_start_logits[feature_index]
            end_logits = all_end_logits[feature_index]
            # This is what will allow us to map some the positions in our logits to span of texts in the original
            # context.
            offset_mapping = features[feature_index]["offset_mapping"]

            # Update minimum null prediction.
            cls_index = features[feature_index]["input_ids"].index(tokenizer.cls_token_id)
            feature_null_score = start_logits[cls_index] + end_logits[cls_index]
            if min_null_score is None or min_null_score < feature_null_score:
                min_null_score = feature_null_score

            # Go through all possibilities for the `n_best_size` greater start and end logits.
            start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
            end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
                    # to part of the input_ids that are not in the context.
                    if (
                        start_index >= len(offset_mapping)
                        or end_index >= len(offset_mapping)
                        or offset_mapping[start_index] is None
                        or offset_mapping[end_index] is None
                    ):
                        continue
                    # Don't consider answers with a length that is either < 0 or > max_answer_length.
                    if end_index < start_index or end_index - start_index + 1 > max_answer_length:
                        continue

                    start_char = offset_mapping[start_index][0]
                    end_char = offset_mapping[end_index][1]
                    valid_answers.append(
                        {
                            "score": start_logits[start_index] + end_logits[end_index],
                            "text": context[start_char: end_char]
                        }
                    )
        
        if len(valid_answers) > 0:
            best_answer = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[0]
        else:
            # In the very rare edge case we have not a single non-null prediction, we create a fake prediction to avoid
            # failure.
            best_answer = {"text": "", "score": 0.0}
        
        # Let's pick our final answer: the best one or the null answer (only for squad_v2)
        predictions[example["id"]] = best_answer["text"]

    return predictions

In [ ]:
final_predictions = postprocess_qa_predictions(valid_dataset, validation_features, raw_predictions.predictions)

Post-processing 64 example predictions split into 744 features.


  0%|          | 0/64 [00:00<?, ?it/s]

In [ ]:
references = [{"id": ex["id"], "answer": ex["answers"]['text'][0]} for ex in valid_dataset]

In [ ]:
def jaccard(row): 
    str1 = row[0]
    str2 = row[1]
    a = set(str1.lower().split()) 
    b = set(str2.lower().split())
    c = a.intersection(b)
    return float(len(c)) / (len(a) + len(b) - len(c))

In [ ]:
res = pd.DataFrame(references)
res['prediction'] = res['id'].apply(lambda r: final_predictions[r])
res['jaccard'] = res[['answer', 'prediction']].apply(jaccard, axis=1)
res

,id,answer,prediction,jaccard
0,312,14,14,1.0
1,252,28 రోజులు,[21] లేదా సెకనుకు 9.192 × 1010 tnt megatonsEn,0.0
2,341,ఆజాద్ హింద్ ఫౌజ్,ఆజాద్ హింద్ ఫౌజ్,1.0
3,327,ఒక నక్క,ఒక నక్క,1.0
4,270,1965,1965,1.0
...,...,...,...,...
59,189,"నవంబర్ 19, 1917","నవంబర్ 19, 1917",1.0
60,72,హిమాలయాలు,హిమాలయాలు,1.0
61,107,22 నెలలు,22 నెలలు,1.0
62,271,ఆరు,ఆరు,1.0


In [ ]:
res.jaccard.mean()

0.6385416666666667

In [ ]:
test_dataset = Dataset.from_pandas(test)

In [ ]:
test_features = test_dataset.map(
    prepare_validation_features,
    batched=True,
    remove_columns=test_dataset.column_names
)

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
test_feats_small = test_features.map(lambda example: example, remove_columns=['example_id', 'offset_mapping'])
test_feats_small

  0%|          | 0/68 [00:00<?, ?ex/s]

Dataset({
    features: ['attention_mask', 'input_ids'],
    num_rows: 68
})

In [ ]:
%%time
test_predictions = trainer.predict(test_feats_small)

***** Running Prediction *****
  Num examples = 68
  Batch size = 4


Wall time: 3min 19s


In [ ]:
%%time
test_features.set_format(type=test_features.format["type"], columns=list(test_features.features.keys()))

Wall time: 1 ms


In [ ]:
%%time
final_test_predictions = postprocess_qa_predictions(test_dataset, test_features, test_predictions.predictions)

Post-processing 7 example predictions split into 68 features.


  0%|          | 0/7 [00:00<?, ?it/s]

Wall time: 490 ms


In [ ]:
sub = pd.read_csv("newtest1.csv")
sub

,id,context,question
0,22bff3dec,"జ్వాలా గుత్తా (జననం 7 సెప్టెంబర్ 1983; వార్ధా,...",జ్వాలా గుత్తా తల్లి పేరు ఏమిటి?
1,282758170,Google Maps (గతంలో Google లోకల్) అనేది Google ...,Google Maps ఎప్పుడు ప్రారంభించబడింది?
2,d60987e0e,గుస్తావ్ రాబర్ట్ కిర్చోఫ్ (12 మార్చి 1824 - 17...,గుస్తావ్ కిర్చోఫ్ ఎప్పుడు జన్మించాడు?
3,f99c770dc,అల్యూమినియం (ఇంగ్లీష్: అల్యూమినియం; ఉత్తర అమెర...,అల్యూమినియం పరమాణు సంఖ్య ఎంత?
4,40dec1964,సహకార ఉద్యమ చరిత్ర ఇంగ్లాండ్‌లో మొట్టమొదటి సహక...,భారతదేశంలో హరిత విప్లవ పితామహుడిగా ఎవరు పరిగణి...
5,307200156,ఆవుకు 4 కాళ్లు ఉంటాయి,ఆవుకు ఎన్ని కాళ్లు ఉన్నాయి?
6,39dec1965,పవన్ కళ్యాణ్ (జననం కొణిదెల కళ్యాణ్ బాబు;[1] 2 ...,పవన్ కళ్యాణ్ ఎవరు?


In [ ]:
sub['PredictionString'] = sub['id'].apply(lambda r: final_test_predictions[r])
sub

,id,context,question,PredictionString
0,22bff3dec,"జ్వాలా గుత్తా (జననం 7 సెప్టెంబర్ 1983; వార్ధా,...",జ్వాలా గుత్తా తల్లి పేరు ఏమిటి?,యెలన్ గుత్తా
1,282758170,Google Maps (గతంలో Google లోకల్) అనేది Google ...,Google Maps ఎప్పుడు ప్రారంభించబడింది?,జూన్ 2005
2,d60987e0e,గుస్తావ్ రాబర్ట్ కిర్చోఫ్ (12 మార్చి 1824 - 17...,గుస్తావ్ కిర్చోఫ్ ఎప్పుడు జన్మించాడు?,(12 మార్చి 1824
3,f99c770dc,అల్యూమినియం (ఇంగ్లీష్: అల్యూమినియం; ఉత్తర అమెర...,అల్యూమినియం పరమాణు సంఖ్య ఎంత?,13.
4,40dec1964,సహకార ఉద్యమ చరిత్ర ఇంగ్లాండ్‌లో మొట్టమొదటి సహక...,భారతదేశంలో హరిత విప్లవ పితామహుడిగా ఎవరు పరిగణి...,"స్వామినాథన్, వర్గీస్ కురియన్"
5,307200156,ఆవుకు 4 కాళ్లు ఉంటాయి,ఆవుకు ఎన్ని కాళ్లు ఉన్నాయి?,4
6,39dec1965,పవన్ కళ్యాణ్ (జననం కొణిదెల కళ్యాణ్ బాబు;[1] 2 ...,పవన్ కళ్యాణ్ ఎవరు?,"ఒక భారతీయ నటుడు, దర్శకుడు, స్క్రీన్ రైటర్, స్..."
